In [73]:
import numpy as np
import json
from PIL import Image
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications import ResNet50
from keras import activations, losses, optimizers
import numpy as np
import shapely.wkt
import shapely
from shapely.geometry import Polygon

In [12]:
input_size = (1024, 1024, 3)

In [19]:
base = Sequential()

base.add(Conv2D(filters = 64, kernel_size = (7, 7), strides = (2, 2), 
                activation = activations.relu, input_shape = input_size))
base.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

base.add(Conv2D(filters = 128, kernel_size = (5, 5), strides = (2, 2), 
                activation = activations.relu))
base.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
base.add(Flatten())

base.add(Dense(units = 5, activation = activations.softmax))

optimizer = optimizers.Adam(learning_rate = 0.01)

base.compile(optimizer = optimizer, loss = losses.categorical_crossentropy, metrics = ['accuracy'])

In [21]:
base.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 509, 509, 64)      9472      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 125, 125, 128)     204928    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 492032)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 2460165   
Total params: 2,674,565
Trainable params: 2,674,565
Non-trainable params: 0
____________________________________________

In [23]:
ResNet50?

In [35]:
# resnet = ResNet50(input_shape = input_size, include_top = False, weights = 'imagenet')
resnet = ResNet50()

In [36]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [41]:
resnet.layers

In [44]:
resnet.trainable = False
model = Model(inputs = resnet.input, outputs = resnet.layers[-2].output)

In [38]:
resnet.layers[-2].output

<tf.Tensor 'avg_pool/Mean:0' shape=(?, 2048) dtype=float32>

In [145]:
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
manhattan_distances(pre_op, post_op)

array([[299.52117136]])

In [158]:
eu_dis = []
man_dis = []
label = []



def process_img(img_array, polygon_pts, scale_pct):
    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
#     xmin = max(int(xmin - (xdiff * scale_pct)), 0)
#     xmax = min(int(xmax + (xdiff * scale_pct)), width)
#     ymin = max(int(ymin - (ydiff * scale_pct)), 0)
#     ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]

base = '/home/abhirag/ONLY_PORTUGAL/images/'

images_list = [base + i for i in os.listdir(base)]




################################################################################################



for i in images_list:
    if i.split('_')[3] == 'pre':

        print('PROCESSING +++++++ ', i)
        pre_image_to_process = i
        post_image_to_process = i.replace('_pre_', '_post_')

        pre_img_arr = np.array(Image.open(pre_image_to_process))
        post_img_arr = np.array(Image.open(post_image_to_process))

        # PROCESS THE ABOVE IMAGE USING SPACENET INFERENCE AND GET OUTPUT JSON

        os.system('python3 /home/abhirag/experiments_on_portugal/spacenet/src/models/inference.py --input ' + i + ' --weights /home/abhirag/experiments_on_portugal/spacenet/models/logs/model_iter_395 --mean /home/abhirag/experiments_on_portugal/spacenet_gt/dataSet/mean.npy --output /home/abhirag/experiments_on_portugal/output_json.json')


        data = json.load(open('/home/abhirag/experiments_on_portugal/output_json.json'))



        # POLYGON IS IN HERE == data['features']['xy'][0]['wkt']

        # OUTPUT OF ABOVE COMMAND IS 'POLYGON ((645.0 944.0,645.0 945.0,643.0 947.0,645.0 947.0,645.0 944.0))'

        # polygon_pts IS OUTPUT POLYGON FROM SEGMENTATION MODEl

        # WILL HAVE TO ITERATE OVER ALL POLYGONS IN TRAINING JSON TO CALCULATE MAX IOU OVER ALL POLYGONS

        original_json = json.load(open('/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000029_post_disaster.json'))

        # THIS DICTIONARY IS OF THE FORM {'uid' : {'IOU' : some percentage, 'damage' : level_of_damage}}

        max_ious = {}

        for x in range(len(data['features']['xy'])):
            max_ious[data['features']['xy'][x]['properties']['uid']] = {'max' : 0, 'damage' : None}
            polygon_geom = shapely.wkt.loads(data['features']['xy'][x]['wkt'])

            max = 0
            d = None

            for i in range(len(original_json['features']['xy'])):
                polygon_original = shapely.wkt.loads(original_json['features']['xy'][i]['wkt'])
                try:
                    if polygon_geom.intersection(polygon_original):
                        iou = polygon_geom.intersection(polygon_original).area / polygon_geom.union(polygon_original).area
                        if iou > max:
                            max = iou
                            d = original_json['features']['xy'][i]['properties']['subtype']
                        # print(iou)
                except:
                    pass

            if max == 0:
                del max_ious[data['features']['xy'][x]['properties']['uid']]
            else:
                max_ious[data['features']['xy'][x]['properties']['uid']]['max'] = max
                max_ious[data['features']['xy'][x]['properties']['uid']]['damage'] = d


        list_of_polygons = list(max_ious.keys())


        for i in range(len(data['features']['xy'])):
            if data['features']['xy'][i]['properties']['uid'] in list_of_polygons:

                poly = np.array(list(shapely.wkt.loads(data['features']['xy'][i]['wkt']).exterior.coords))

                pre_x = process_img(pre_img_arr, poly, 1.0)
                post_x = process_img(post_img_arr, poly, 1.0)

        #         print('pre = ', pre_x.shape)
        #         print('post = ', post_x.shape)

                pre = np.array(Image.fromarray(pre_x, 'RGB').resize((224, 224)))
                post = np.array(Image.fromarray(post_x, 'RGB').resize((224, 224)))

        #         print(pre.shape)
                eu_dis.append(euclidean_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3)))[0][0])
                man_dis.append(manhattan_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3)))[0][0])
                label.append(max_ious[data['features']['xy'][i]['properties']['uid']]['damage'])


        #         Image.fromarray(post_x, 'RGB').show()

        #         print(poly)
        #         break

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001402_pre_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001159_pre_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001700_pre_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 725 542 at 725 542
TopologyException: I

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001620_pre_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 229 707 at 229 707
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 229 707 at 229 707
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 65 761 at 65 761
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 65 761 at 65 761
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 387 661 at 387 661
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 387 661 at 387 661
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 336 611 at 336 611
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 336 611 at 336 611
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 336 611 at 336 611
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 336 611 at 336 611
TopologyException: Input

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00000954_pre_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00000469_pre_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001860_pre_disaster.png


...

TopologyException: Input geom 0 is invalid: Self-intersection at or near point 790 855 at 790 855
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 275 0 at 275 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 275 0 at 275 0


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001439_pre_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 22 0 at 22 0
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 2

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00001600_pre_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00000237_pre_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 98 211 at 98 211
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 98 211 at 98 211


In [170]:
for i in set(label):
    print('   ', i, label.count(i))

    un-classified 413
    no-damage 13259
    destroyed 183
    major-damage 127


In [181]:
eu_dis

[19.416725,
 12.570707,
 11.832879,
 9.189579,
 9.87831,
 6.639532,
 14.218818,
 14.281766,
 7.3988423,
 9.805011,
 8.271767,
 10.328487,
 12.394597,
 8.021404,
 5.8201704,
 7.7354693,
 7.67007,
 7.865192,
 12.365219,
 8.071925,
 7.187467,
 13.6381,
 12.967588,
 6.427815,
 7.0282426,
 19.269312,
 14.147797,
 11.531903,
 12.253088,
 9.513928,
 18.91764,
 16.330704,
 9.019599,
 8.992559,
 6.719427,
 6.4360204,
 8.51394,
 14.042788,
 18.12699,
 11.595563,
 5.2617064,
 14.693761,
 14.182705,
 18.248203,
 16.228416,
 9.854342,
 7.6167254,
 13.786029,
 22.380835,
 21.8018,
 17.632294,
 19.63168,
 17.712225,
 21.251442,
 6.4875712,
 21.163214,
 30.429043,
 18.030619,
 19.884497,
 17.995712,
 15.696833,
 7.725019,
 10.770538,
 9.120874,
 24.249704,
 11.860836,
 9.244706,
 11.872515,
 7.290311,
 12.867008,
 11.476432,
 11.476021,
 11.750107,
 6.1630907,
 7.6586537,
 11.73025,
 12.228576,
 13.714767,
 26.572412,
 15.628126,
 16.865831,
 10.163777,
 4.441817,
 12.345174,
 16.283405,
 13.821592,
 

In [182]:
[i ** 2 for i in eu_dis]

[377.0092158881598,
 158.02268255444142,
 140.0170270016397,
 84.44836238121206,
 97.58101247760442,
 44.08338636396002,
 202.17477709513423,
 203.96883830225215,
 54.74286789444909,
 96.13823670170041,
 68.42212372046197,
 106.67765189429338,
 153.62603611932082,
 64.34292640433705,
 33.87438351444962,
 59.837485929853074,
 58.829976433528145,
 61.86124419787325,
 152.8986437919084,
 65.15597584141597,
 51.65968328822623,
 185.9977626200416,
 168.1583495519626,
 41.316805166167114,
 49.39619387638436,
 371.30638128859937,
 200.16014950815588,
 132.9847929572943,
 150.138165472923,
 90.51483385513075,
 357.8771291259327,
 266.6918844918255,
 81.3531654150438,
 80.8661251557387,
 45.15070067000124,
 41.4223582583802,
 72.48717189683612,
 197.1998822272326,
 328.58774342519246,
 134.45707977665825,
 27.685553737138434,
 215.9066085602035,
 201.14911900485458,
 332.99692286016943,
 263.3615002432489,
 97.10806533142113,
 58.014506501367805,
 190.05459178388355,
 500.9017564735004,
 475.31

In [179]:
man_dis

[408.1756925059817,
 123.0899410170241,
 126.4508931891105,
 155.32118192985448,
 103.35875135304559,
 68.39229472351144,
 130.79520444447553,
 324.4057776522386,
 68.78573229893664,
 100.10010440725091,
 129.59584390270174,
 204.84885743090854,
 128.55137758834462,
 89.92501434747868,
 65.49648666608937,
 81.4162183152983,
 81.44743285149525,
 82.81271683510931,
 127.94536113201411,
 74.7709222280173,
 66.8458185941272,
 141.57334858251488,
 257.6538853251759,
 57.20498046269967,
 68.85768156802169,
 257.4593078522885,
 172.4426448686354,
 98.0478322530289,
 130.57892433974484,
 88.05409543972377,
 227.24875256270025,
 263.7679008892601,
 91.55962176563116,
 116.18511491728896,
 76.78294783183617,
 77.45181010507804,
 92.3798424720153,
 140.6981615211771,
 359.3221788706687,
 216.31399796975165,
 55.93871717890033,
 144.8536480138573,
 314.9172050057241,
 443.434476836489,
 191.26221782787616,
 117.98954718270306,
 76.66999400803252,
 139.90220245976525,
 315.9877577491152,
 410.68961

In [183]:
mlp_data = {}
for ind, i, j, k in zip(range(len(man_dis)), eu_dis, man_dis, label):
    mlp_data[ind] = {'man_dis' : i, 'eu_dis' : j, 'label' : k}

In [186]:
for i in list(mlp_data.keys()):
    if mlp_data[i]['label'] == 'un-classified':
        del mlp_data[i]

In [187]:
c = 0
for i in list(mlp_data.keys()):
    if mlp_data[i]['label'] == 'no-damage':
        
        c += 1
        del mlp_data[i]
        if c == 13059:
            break

In [188]:
len(mlp_data)

510

In [192]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

mlpc = MLPClassifier()
mlpc.fit(np.array([mlp_data[i]['man_dis'] for i in list(mlp_data.keys())]).reshape(-1, 1), [mlp_data[i]['label'] for i in list(mlp_data.keys())])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [193]:
print(classification_report([mlp_data[i]['label'] for i in list(mlp_data.keys())], mlpc.predict(np.array([mlp_data[i]['man_dis'] for i in list(mlp_data.keys())]).reshape(-1, 1))))

              precision    recall  f1-score   support

   destroyed       0.35      0.25      0.29       183
major-damage       0.00      0.00      0.00       127
   no-damage       0.40      0.76      0.52       200

    accuracy                           0.39       510
   macro avg       0.25      0.34      0.27       510
weighted avg       0.28      0.39      0.31       510



/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [200]:
print(classification_report(mlpc.predict(np.array(man_dis).reshape(-1, 1)), label))

               precision    recall  f1-score   support

    destroyed       0.00      0.00      0.00       204
 major-damage       0.99      0.01      0.02     13778
    no-damage       0.00      0.00      0.00         0
un-classified       0.00      0.00      0.00         0

     accuracy                           0.01     13982
    macro avg       0.25      0.00      0.00     13982
 weighted avg       0.98      0.01      0.02     13982



/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [195]:
mlpc.predict(np.array([mlp_data[i]['man_dis'] for i in list(mlp_data.keys())]).reshape(-1, 1))

array(['no-damage', 'no-damage', 'no-damage', 'destroyed', 'no-damage',
       'no-damage', 'destroyed', 'no-damage', 'no-damage', 'no-damage',
       'destroyed', 'destroyed', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'destroyed', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'destroyed', 'destroyed', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'destroyed', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-damage',
       'no-damage', 'no-damage', 'no-damage', 'no-damage', 'no-d

In [142]:
max_ious

{'dc69b6c4-d616-4a03-893c-a82a72c25a69': {'max': 0.6190662115067779,
  'damage': 'no-damage'},
 'c319be70-2d2f-4d52-90f1-e2d99fdd8d44': {'max': 0.00119702212143351,
  'damage': 'no-damage'},
 'e2ee8186-66ef-4dae-8e91-d228ee8037e1': {'max': 0.005717622495390479,
  'damage': 'no-damage'},
 '5e7d221c-7fa7-414f-b42c-77191b9c4930': {'max': 0.005717622495390479,
  'damage': 'no-damage'},
 'fb7612fc-b5f1-49c8-a65c-caff3c51d35c': {'max': 0.005717622495390479,
  'damage': 'no-damage'},
 '89560488-b6fb-42ea-a9f9-dbd87e574222': {'max': 0.005717622495390479,
  'damage': 'no-damage'},
 '535315f0-ce19-49fa-94d4-4a78a7a84ec7': {'max': 0.6152919328496816,
  'damage': 'no-damage'},
 'a6cf431f-0daf-4ba4-bc26-420c2be88b3b': {'max': 0.001721176206372871,
  'damage': 'no-damage'},
 'c1a3628b-e52b-47df-84ec-671c65eb29a8': {'max': 0.03145148599284689,
  'damage': 'no-damage'},
 '51393213-9956-493f-a3d6-d22be4c1defa': {'max': 0.00849242339244237,
  'damage': 'no-damage'},
 '5bb0715f-4d4b-4c9f-856d-b5e1ad688af

In [146]:
for i in range(len(data['features']['xy'])):
    if data['features']['xy'][i]['properties']['uid'] in list_of_polygons:

        poly = np.array(list(shapely.wkt.loads(data['features']['xy'][i]['wkt']).exterior.coords))

        pre_x = process_img(pre_img_arr, poly, 1.0)
        post_x = process_img(post_img_arr, poly, 1.0)

#         print('pre = ', pre_x.shape)
#         print('post = ', post_x.shape)

        pre = np.array(Image.fromarray(pre_x, 'RGB').resize((224, 224)))
        post = np.array(Image.fromarray(post_x, 'RGB').resize((224, 224)))
        
#         print(pre.shape)
        print(euclidean_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3))))
        print(manhattan_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3))))
        print(max_ious[data['features']['xy'][i]['properties']['uid']]['damage'])
        
#         Image.fromarray(post_x, 'RGB').show()

#         print(poly)
#         break

[[10.187054]]
[[136.49258562]]
no-damage
[[9.832173]]
[[89.93986817]]
no-damage
[[5.711263]]
[[63.94303158]]
no-damage
[[4.0592194]]
[[40.66971117]]
no-damage
[[4.8145657]]
[[53.02138454]]
no-damage
[[6.2640934]]
[[57.26699246]]
no-damage
[[10.149529]]
[[198.3178396]]
no-damage
[[10.433881]]
[[107.30941628]]
no-damage
[[12.811852]]
[[127.15179386]]
no-damage
[[11.672188]]
[[102.25228642]]
no-damage
[[11.449026]]
[[110.26617709]]
no-damage
[[6.084373]]
[[65.57760848]]
no-damage
[[10.301892]]
[[100.76173162]]
no-damage
[[7.214843]]
[[75.42208295]]
no-damage
[[13.31432]]
[[142.93408467]]
no-damage
[[11.587831]]
[[278.66164191]]
no-damage
[[17.316658]]
[[260.49407182]]
no-damage
[[12.487604]]
[[253.46584404]]
no-damage
[[11.793833]]
[[255.16665728]]
no-damage
[[8.946565]]
[[86.54315932]]
no-damage
[[13.678971]]
[[308.34729266]]
no-damage
[[7.9394765]]
[[96.85947458]]
un-classified
[[8.202332]]
[[84.49843481]]
un-classified
[[13.561442]]
[[136.49807566]]
no-damage
[[14.628953]]
[[239.137674

KeyboardInterrupt: 

In [3]:
import os
import json

base = '/home/abhirag/ONLY_PORTUGAL/labels/'

l = [base + i for i in os.listdir(base)]

d = []

for i in range(len(l)):
    try:
        
        data = json.load(open(l[i]))
        for i in range(len(data['features']['xy'])):
            if data['features']['xy'][i]['properties']['subtype'] == 'destroyed':
                print(l[i])
#         print(data['features']['xy'][3])
    except:
        pass
#     break

for i in set(d):
    print('counts of ', i, ' = ', d.count(i))

/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000513_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000228_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001212_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000018_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001107_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001642_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000035_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000733_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001748_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001849_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001204_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001546_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/label

/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000513_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000228_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001748_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000505_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000483_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001204_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000035_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000648_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000189_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001653_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000131_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001846_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/

/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000549_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001846_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001830_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000576_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001376_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000131_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001642_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001204_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000035_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001227_post_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000100_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001350_pre_disaster.json
/home/abhirag/ONLY_PORTUGAL/labels

In [141]:
original_json['features']['xy']

[{'properties': {'feature_type': 'building',
   'subtype': 'un-classified',
   'uid': '7074b1de-2759-474e-aae3-c863e6b36b5f'},
  'wkt': 'POLYGON ((0.007983494621192296 1.620363670815105, 3.701073031552089 0.0003437376499227781, 45.83381039709077 0.001537897701697094, 54.70058066357948 6.31796359281019, 37.31021116341804 20.77737194394657, 23.04620022545694 8.076540287730586, 18.16126497331149 9.248924751010851, 13.27632972038067 8.076540289824887, 7.018204022910634 15.16118712476813, 0.00798761908554565 6.278299212596537, 0.007983494621192296 1.620363670815105))'},
 {'properties': {'feature_type': 'building',
   'subtype': 'no-damage',
   'uid': '57213a7a-9ea2-4936-adfd-277480658c01'},
  'wkt': 'POLYGON ((100.2281772188318 97.17775929597775, 121.5264949205002 100.4995152689889, 123.2850716114924 114.1773339779385, 110.1934451341928 116.5221029003104, 100.2281772188318 97.17775929597775))'},
 {'properties': {'feature_type': 'building',
   'subtype': 'no-damage',
   'uid': 'caea1a25-f175

In [152]:

base = '/home/abhirag/ONLY_PORTUGAL/images/'

images_list = [base + i for i in os.listdir(base)]
cnt = 0
for i in images_list:
    if i.split('_')[3] == 'pre':
        print('yes')
        cnt += 1

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [153]:
cnt

1500

In [ ]:

pre_image_to_process = '/home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00000029_pre_disaster.png'
post_image_to_process = '/home/abhirag/ONLY_PORTUGAL/images/portugal-wildfire_00000029_post_disaster.png'

pre_img_arr = np.array(Image.open(pre_image_to_process))
post_img_arr = np.array(Image.open(post_image_to_process))

# PROCESS THE ABOVE IMAGE USING SPACENET INFERENCE AND GET OUTPUT JSON

def process_img(img_array, polygon_pts, scale_pct):
    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
#     xmin = max(int(xmin - (xdiff * scale_pct)), 0)
#     xmax = min(int(xmax + (xdiff * scale_pct)), width)
#     ymin = max(int(ymin - (ydiff * scale_pct)), 0)
#     ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]


data = json.load(open('/home/abhirag/experiments_on_portugal/output_json.json'))



# POLYGON IS IN HERE == data['features']['xy'][0]['wkt']

# OUTPUT OF ABOVE COMMAND IS 'POLYGON ((645.0 944.0,645.0 945.0,643.0 947.0,645.0 947.0,645.0 944.0))'

# polygon_pts IS OUTPUT POLYGON FROM SEGMENTATION MODEl

# WILL HAVE TO ITERATE OVER ALL POLYGONS IN TRAINING JSON TO CALCULATE MAX IOU OVER ALL POLYGONS

original_json = json.load(open('/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00000029_post_disaster.json'))

# THIS DICTIONARY IS OF THE FORM {'uid' : {'IOU' : some percentage, 'damage' : level_of_damage}}

max_ious = {}

for x in range(len(data['features']['xy'])):
    max_ious[data['features']['xy'][x]['properties']['uid']] = {'max' : 0, 'damage' : None}
    polygon_geom = shapely.wkt.loads(data['features']['xy'][x]['wkt'])

    max = 0
    d = None

    for i in range(len(original_json['features']['xy'])):
        polygon_original = shapely.wkt.loads(original_json['features']['xy'][i]['wkt'])
        try:
            if polygon_geom.intersection(polygon_original):
                iou = polygon_geom.intersection(polygon_original).area / polygon_geom.union(polygon_original).area
                if iou > max:
                    max = iou
                    d = original_json['features']['xy'][i]['properties']['subtype']
                # print(iou)
        except:
            pass

    if max == 0:
        del max_ious[data['features']['xy'][x]['properties']['uid']]
    else:
        max_ious[data['features']['xy'][x]['properties']['uid']]['max'] = max
        max_ious[data['features']['xy'][x]['properties']['uid']]['damage'] = d


list_of_polygons = list(max_ious.keys())


for i in range(len(data['features']['xy'])):
    if data['features']['xy'][i]['properties']['uid'] in list_of_polygons:

        poly = np.array(list(shapely.wkt.loads(data['features']['xy'][i]['wkt']).exterior.coords))

        pre_x = process_img(pre_img_arr, poly, 1.0)
        post_x = process_img(post_img_arr, poly, 1.0)

#         print('pre = ', pre_x.shape)
#         print('post = ', post_x.shape)

        pre = np.array(Image.fromarray(pre_x, 'RGB').resize((224, 224)))
        post = np.array(Image.fromarray(post_x, 'RGB').resize((224, 224)))
        
#         print(pre.shape)
        eu_dis.append(euclidean_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3)))[0][0])
        man_dis.append(manhattan_distances(model.predict(pre.reshape(1, 224, 224, 3)), model.predict(post.reshape(1, 224, 224, 3)))[0][0])
        label.append(max_ious[data['features']['xy'][i]['properties']['uid']]['damage'])
        
        
#         Image.fromarray(post_x, 'RGB').show()

#         print(poly)
#         break